---
# 신경망 종합 테스트
- burst 하여 active cell 이 많으면 새로운 패턴을 인지한다는 의미
- 먼저 spatial pooler 가 안정되어야 한다.
---
encoder - sp - tm -> prediction + textual representation<br/>
예측 cell / 활성화 cell 비교 - 예측이 맞는지 확인 가능<br/>
예측 column / 입력 column 비교 - 마찬가지<br/>
burst 한 column / 활성화 된 column - 마찬가지<br/>


In [1]:
import numpy as np
import random
import nbimporter
import TM
import Encoder
import SP3

Importing Jupyter notebook from TM.ipynb
Importing Jupyter notebook from memoryStruct.ipynb
Importing Jupyter notebook from SP3.ipynb


In [2]:
encoder = Encoder.ScalarEncoder(out_size=100, w=20, max_val=30)
sp = SP3.SpatialPooler(encoder.out_size, columns=512, perm_conn=.6, minOver=encoder.w/5, potential_rate=0.9)

In [3]:
for epoch in range(3):
    a = np.arange(encoder.max_val)
    random.shuffle(a)
    
    for i in a:
        input_data = encoder.encode(i)
        sp.compute_SDR(input_data)

In [6]:
for i in range(encoder.max_val):
    sp.compute_SDR(encoder.encode(i), learn=False)
    sdr1 = sp.getActiveColumns()
    sp.compute_SDR(encoder.encode(i+1), learn=False)
    sdr2 = sp.getActiveColumns()
    
    sp.similarity(sdr1, sdr2, False)
    print('')

유사도 : 0.7
[154 172 206 212 217 253 282 374 464 492]
[154 206 212 253 282 314 332 374 386 388 464]

유사도 : 0.36363636363636365
[154 206 212 253 282 314 332 374 386 388 464]
[ 90 111 183 199 212 214 253 290 314 317 386 430]

유사도 : 0.25
[ 90 111 183 199 212 214 253 290 314 317 386 430]
[ 90 201 290 297 325 382 399 430 449 482]

유사도 : 0.4
[ 90 201 290 297 325 382 399 430 449 482]
[154 156 164 201 209 297 382 398 465 482]

유사도 : 0.0
[154 156 164 201 209 297 382 398 465 482]
[104 195 224 296 318 367 389 401 475 486]

유사도 : 0.4
[104 195 224 296 318 367 389 401 475 486]
[ 14 192 224 302 318 355 359 389 401 455]

유사도 : 0.5
[ 14 192 224 302 318 355 359 389 401 455]
[ 73 156 219 224 339 355 389 401 455 486]

유사도 : 0.1
[ 73 156 219 224 339 355 389 401 455 486]
[ 17  73  83 117 127 176 191 192 255 378]

유사도 : 0.2
[ 17  73  83 117 127 176 191 192 255 378]
[ 57  83  94 148 255 358 380 413 433 497]

유사도 : 0.3
[ 57  83  94 148 255 358 380 413 433 497]
[ 17  83 133 240 255 292 366 413 442 503 507]

유사도 :

---
# 테스트
- 정수 1, 2, 3, 4, 5 순서 학습
- burst 가 적게 되면 안정적으로 학습

- 이후에 1, 2, 4, 3, 5 처럼 데이터를 조작하여 학습
---

In [40]:
tm = TM.TemporalMemory(sdr_size=sp.columnCount, max_seg=5, threshold=7)

In [43]:
tm.learn = True

seq_data = [1,2,3,4,5]

for i in range(10):
    for s in seq_data:
        dat = encoder.encode(s)
        sp.compute_SDR(dat, learn=False)
        sdr = sp.getActiveColumns_raw()
        tm.compute(sdr)

In [44]:
tm.learn = False

# 검증 시 학습기능 off
for d in seq_data:
    dat = encoder.encode(d)
    sp.compute_SDR(dat, learn=False)
    sdr = sp.getActiveColumns_raw()
    tm.compute(sdr)
    tm.test(prev_pred, sp.getActiveColumns())
    prev_pred = tm.column_predicted()
    
    print("현재 columns (sdr) : {}".format(sp.getActiveColumns()))
    print("다음 columns (예측) : {}".format(tm.column_predicted()))
    print("현재 burst : {}".format(tm.column_bursted()))
    print("\n")
    

Loading BokehJS ...

예측 : 0.5333333333333333
현재 columns (sdr) : [ 48  62 147 208 210 286 315 396 397 411 421 427 461 482 510]
다음 columns (예측) : [482, 327, 457, 395, 427, 397, 433, 404, 286, 95]
현재 burst : [210]




Loading BokehJS ...

예측 : 1.0
현재 columns (sdr) : [ 95 286 327 395 397 404 427 433 457 482]
다음 columns (예측) : [482, 483, 418, 327, 135, 71, 392, 427, 395, 397, 463, 340, 188, 286, 95]
현재 burst : []




Loading BokehJS ...

예측 : 1.0
현재 columns (sdr) : [ 71  95 135 188 286 327 340 392 395 397 418 427 463 482 483]
다음 columns (예측) : [482, 483, 418, 71, 296, 327, 39, 392, 135, 395, 397, 463, 497, 344, 347, 188, 95]
현재 burst : [418, 188]




Loading BokehJS ...

예측 : 1.0
현재 columns (sdr) : [ 39  71 135 188 296 327 344 392 418 463 497]
다음 columns (예측) : [418, 135, 296, 77, 463, 273, 242, 20, 503, 347, 445, 254]
현재 burst : []




Loading BokehJS ...

예측 : 1.0
현재 columns (sdr) : [ 20  77 242 254 273 296 347 418 445 503]
다음 columns (예측) : [482, 421, 427, 396, 397, 461, 208, 48, 210, 147, 411, 62, 315, 510, 286]
현재 burst : []




---
# 데이터 변경

- 새로운 시퀀스를 학습
- 이전과 변화가 생긴 부분 (3->4, 4->5) 에서 예측 column 이 대략 2 배 되었는데 이전의 배웠던 시퀀스까지 고려하는 것임
- Temporal Memory 의 Segment 제한, synapse 제한을 조절하면 이전에 학습한 시퀀스의 망각(Segment, synapse 파괴) 시점을 조절할 수 있다.

---

In [45]:
tm.learn = True

test_data = [1,2,4,3,5]

for i in range(10):
    for s in test_data:
        dat = encoder.encode(s)
        sp.compute_SDR(dat, learn=False)
        sdr = sp.getActiveColumns_raw()
        tm.compute(sdr)

In [46]:
tm.learn = False

# 검증 시 학습기능 off
for d in test_data:
    dat = encoder.encode(d)
    sp.compute_SDR(dat, learn=False)
    sdr = sp.getActiveColumns_raw()
    tm.compute(sdr)
    tm.test(prev_pred, sp.getActiveColumns())
    prev_pred = tm.column_predicted()
    
    print("현재 columns (sdr) : {}".format(sp.getActiveColumns()))
    print("다음 columns (예측) : {}".format(tm.column_predicted()))
    print("현재 burst : {}".format(tm.column_bursted()))
    print("\n")
    

Loading BokehJS ...

예측 : 1.0
현재 columns (sdr) : [ 48  62 147 208 210 286 315 396 397 411 421 427 461 482 510]
다음 columns (예측) : [482, 327, 457, 395, 427, 397, 433, 404, 286, 95]
현재 burst : []




Loading BokehJS ...

예측 : 1.0
현재 columns (sdr) : [ 95 286 327 395 397 404 427 433 457 482]
다음 columns (예측) : [482, 483, 418, 39, 327, 135, 296, 427, 395, 71, 392, 463, 397, 340, 344, 188, 286, 95]
현재 burst : []




Loading BokehJS ...

예측 : 0.9090909090909091
현재 columns (sdr) : [ 39  71 135 188 296 327 344 392 418 463 497]
다음 columns (예측) : [135, 392, 395, 397, 273, 20, 286, 418, 296, 427, 188, 445, 71, 327, 77, 463, 340, 347, 95, 482, 483, 242, 503, 254]
현재 burst : [344, 497, 39]




Loading BokehJS ...

예측 : 1.0
현재 columns (sdr) : [ 71  95 135 188 286 327 340 392 395 397 418 427 463 482 483]
다음 columns (예측) : [135, 392, 395, 397, 273, 20, 418, 39, 296, 188, 445, 71, 327, 77, 463, 340, 344, 347, 95, 482, 497, 242, 503, 254]
현재 burst : [482]




Loading BokehJS ...

예측 : 1.0
현재 columns (sdr) : [ 20  77 242 254 273 296 347 418 445 503]
다음 columns (예측) : [482, 421, 427, 396, 397, 461, 48, 315, 210, 147, 208, 62, 411, 510, 286]
현재 burst : [273]




In [ ]:
tm.learn = False
test_data = [1,2,4,3,5]
# 검증 시 학습기능 off
for d in test_data:
    dat = encoder.encode(d)
    sp.compute_SDR(dat, learn=False)
    sdr = sp.getActiveColumns_raw()
    tm.compute(sdr)
    tm.test(prev_pred, sp.getActiveColumns())
    prev_pred = tm.column_predicted()
    
    print("현재 columns (sdr) : {}".format(sp.getActiveColumns()))
    print("다음 columns (예측) : {}".format(tm.column_predicted()))
    print("현재 burst : {}".format(tm.column_bursted()))
    print("\n")
    

In [15]:
# len(tm.connInfo.totalSegments)
cell = 2000
print('{} 번 cell 에 연결되어 있는 synapse : {}'.format(cell, tm.connInfo.totalCells[cell].connectedSynapses))
#print(len(tm.prev_activeCells))
#print(len(tm.activeCells))
#for c in tm.connInfo.totalCells:
 #   if(len(c.connectedSynapses) > 0):
  #      print("살아있는 cell : " + str(c.cell_id))
        
#choose = list(tm.connInfo.totalCells[cell].connectedSynapses)[0]
#for i in tm.connInfo.totalSegments:
 #   for j in i.synapses:
  #      if (j.synapse_id == choose):
   #         print(j.synapse_id)
    #        print(j.preSynapticCell)
     #       print(j.permanence)
        
print(sp.getActiveColumns())
print(tm.predicted)
print(tm.bursted)

2000 번 cell 에 연결되어 있는 synapse : set()
[ 86 160 176 237 353 361 415 458 492 500]
[131, 285, 329, 361, 176, 415, 86, 361, 458, 492, 237, 353, 285, 462, 40, 285, 361, 40, 145, 451, 462, 500, 40]
[86, 237, 353, 361, 458, 492]
